In [2]:
import tokenizer as to
import copy as co
order = {'+': 0, '-': 0, '*': 1, '/': 1, '^': 2, '(': 3,'->':4, '=': 4,}
operation = {
    '+': lambda a, b: a + b,
    '-': lambda a, b: a - b,
    '*': lambda a, b: a * b,
    '/': lambda a, b: a / b,
    '^': lambda a, b: a ** b,
}

def simple_math(expression):
    expression = co.deepcopy(expression)
    result = []
    operator = []
    number = []
    last = to.token('DUMMY','')
    def operate(a, b, s):
        nonlocal result, operator, number
        number = number[:-2] + [to.token('0' if (a.type != 'X' and b.type != 'X') else 'X', operation[s.value](a.value, b.value))]
        operator.pop()
    def add(cur):
        nonlocal operator, number
        if(
            len(operator) > 0 and 
            order[operator[-1].value] > order[cur.value]
        ):
            if((number[-2].type == '0') and number[-1].type == '0'):
                operate(number[-2], number[-1], operator[-1])
            else:
                if(operator[-1].value not in ['+', '-'] and (number[-2].type in ['0','X']) and number[-1].type in ['0','X']):
                    operate(number[-2], number[-1], operator[-1])
                    return
                if(number[-2].type != '0' and number[-1].type == '0'):
                    number[-1].type = 'X'
                if(number[-1].type != '0' and number[-2].type == '0'):
                    number[-2].type = 'X'
                operator.append(cur)
        else:
            
            operator.append(cur)
    for i in range(len(expression)):
        cur = expression[i]
        if(expression[i].type in to.parentesis.keys()):
            group = simple_math(expression[i].value)
            if(len(group) == 1):
                cur = group[0]
            else:
                expression[i].value = group
        if(cur.type != '#'):
            if(last.type in ['A', '0']):
                add(to.token('#', '*'))
            elif(last.type == 'F'):
                add(to.token('#', '->'))
            number.append(cur)
        else:
            add(cur)
        last = cur
    while(len(operator) > 0):
        if(number[-2].type == '0' and number[-1].type == '0'):
            operate(number[-2], number[-1], operator[-1])
        elif(operator[-1].value not in ['+', '-']):
            if((number[-2].type in ['0','X']) and number[-1].type in ['0','X']):
                operate(number[-2], number[-1], operator[-1])
                continue
            if(number[-2].type != '0' and number[-1].type == '0'):
                number[-1].type = 'X'
            if(number[-1].type != '0' and number[-2].type == '0'):
                number[-2].type = 'X'
            result += [number[-1], operator[-1]]
            operator = operator[:-1]
            number = number[:-1]
        else:
            result += [number[-1], operator[-1]]
            operator = operator[:-1]
            number = number[:-1]
    result += [number[0]]
    result.reverse()
    return result
# exp = to.parse('10 + 10 * 10 * x + 4 + y * 10 + 10 * z')
exp = to.parse('8.5 sind(theta)')
print(simple_math(exp))


[8.5, *, sind, ->, theta]


In [11]:


def parse_system(input):
    expression = []
    for i in input:
        expression.append(to.parse(i))
    return expression






def replace(line):
    i = 0
    while i < len(line):
        if(line[i].type in to.parentesis.keys()):
            replace(line[i].value)
        elif(line[i].type == 'A'):
            if(line[i].value in value.keys()):
                line[i] = value[line[i].value]
            elif(line[i].value in expression.keys()):
                cur = expression[line[i].value][0]
                replace(cur)
                cur[:] = simple_math(cur)
                if(len(cur) == 1):
                    value[line[i].value] = cur[0]
                line[:] = line[:i] + cur + line[i+1:]
                i += len(cur)
                continue
        i += 1
    
    line[:] = simple_math(line)



# for i in literal:
#     replace(i)
input = [
    'f = m a',
    'f = p sind(theta)',
    'p = m g',
    'g = 9.8',
    'm = 8.5',
    'theta = 30'
]


literal = parse_system(input)

expression, value = to.parse_group(literal)

for i in expression.keys():
    for j in expression[i]:
        replace(j)



print(expression)
print(value)


RuntimeError: dictionary changed size during iteration

In [1]:
output = [
    'm a = m g sind(theta)',
    'a = g sind(theta)',
    'a = 9.8 sind(30)',
    'a = 4.9',
]